<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# xDeepFM : the eXtreme Deep Factorization Machine 
This notebook will give you a quick example of how to train an [xDeepFM model](https://arxiv.org/abs/1803.05170). 
xDeepFM \[1\] is a deep learning-based model aims at capturing both lower- and higher-order feature interactions for precise recommender systems. Thus it can learn feature interactions more effectively and manual feature engineering effort can be substantially reduced. To summarize, xDeepFM has the following key properties:
* It contains a component, named CIN, that learns feature interactions in an explicit fashion and in vector-wise level;
* It contains a traditional DNN component that learns feature interactions in an implicit fashion and in bit-wise level.
* The implementation makes this model quite configurable. We can enable different subsets of components by setting hyperparameters like `use_Linear_part`, `use_FM_part`, `use_CIN_part`, and `use_DNN_part`. For example, by enabling only the `use_Linear_part` and `use_FM_part`, we can get a classical FM model.

In this notebook, we test xDeepFM on [Criteo dataset](http://labs.criteo.com/category/dataset).

## 0. Global Settings and Imports

In [1]:
import os
import sys
import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources, prepare_hparams
from recommenders.models.deeprec.models.xDeepFM import XDeepFMModel
from recommenders.models.deeprec.io.iterator import FFMTextIterator

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.7.13 (default, Oct 18 2022, 18:57:03) 
[GCC 11.2.0]
Tensorflow version: 2.7.4


#### Parameters

In [2]:
EPOCHS = 10
BATCH_SIZE = 1024
RANDOM_SEED = 42  # Set this to None for non-deterministic result


xDeepFM uses the FFM format as data input: `<label> <field_id>:<feature_id>:<feature_value>`  
Each line represents an instance, `<label>` is a binary value with 1 meaning positive instance and 0 meaning negative instance. 
Features are divided into fields. For example, user's gender is a field, it contains three possible values, i.e. male, female and unknown. Occupation can be another field, which contains many more possible values than the gender field. Both field index and feature index are starting from 1. <br>

In [3]:
tmpdir = TemporaryDirectory()
data_path = tmpdir.name
yaml_file = os.path.join(data_path, r'xDeepFM.yaml')
output_file = os.path.join(data_path, r'output.txt')
train_file = os.path.join(data_path, r'cretio_tiny_train')
valid_file = os.path.join(data_path, r'cretio_tiny_valid')
test_file = os.path.join(data_path, r'cretio_tiny_test')

if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/deeprec/', data_path, 'xdeepfmresources.zip')


100%|██████████| 10.3k/10.3k [00:01<00:00, 7.08kKB/s]


## 2. Criteo data 

Now let's try the xDeepFM on a real world dataset, a small sample from [Criteo dataset](http://labs.criteo.com/category/dataset). Criteo dataset is a well known industry benchmarking dataset for developing CTR prediction models and it's frequently adopted as evaluation dataset by research papers. 

The original dataset is too large for a lightweight demo, so we sample a small portion from it as a demo dataset.

In [5]:
print('Demo with Criteo dataset')
hparams = prepare_hparams(yaml_file, 
                          FEATURE_COUNT=2300000, 
                          FIELD_COUNT=39, 
                          cross_l2=0.01, 
                          embed_l2=0.01, 
                          layer_l2=0.01,
                          learning_rate=0.002, 
                          batch_size=BATCH_SIZE, 
                          epochs=EPOCHS, 
                          cross_layer_sizes=[20, 10], 
                          init_value=0.1, 
                          layer_sizes=[20,20],
                          use_Linear_part=True, 
                          use_CIN_part=True, 
                          use_DNN_part=True)


Demo with Criteo dataset


In [6]:
model = XDeepFMModel(hparams, FFMTextIterator, seed=RANDOM_SEED)

Add linear part.
Add CIN part.
Add DNN part.


/anaconda/envs/reco_gpu/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:450: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/XDeepFM/embedding/embedding_lookup_sparse/GatherV2_grad/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradients/XDeepFM/embedding/embedding_lookup_sparse/GatherV2_grad/Reshape:0", shape=(None, 10), dtype=float32), dense_shape=Tensor("gradients/XDeepFM/embedding/embedding_lookup_sparse/GatherV2_grad/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
2022-11-16 11:23:52.751265: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

In [7]:
# check the predictive performance before the model is trained
print(model.run_eval(test_file)) 

2022-11-16 11:23:55.611091: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8401
2022-11-16 11:23:56.047993: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


{'auc': 0.4728, 'logloss': 0.7113}


In [ ]:
%%time
model.fit(train_file, valid_file)

at epoch 1
train info: logloss loss:358.075682867141
eval info: auc:0.729, logloss:0.5033
at epoch 1 , train time: 27.8 eval time: 4.1
at epoch 2
train info: logloss loss:20.122149774006434
eval info: auc:0.74, logloss:0.4971
at epoch 2 , train time: 27.5 eval time: 4.1
at epoch 3
train info: logloss loss:1.1374033954408433
eval info: auc:0.7419, logloss:0.4959
at epoch 3 , train time: 27.9 eval time: 4.1
at epoch 4
train info: logloss loss:0.5466613788453359
eval info: auc:0.742, logloss:0.4961
at epoch 4 , train time: 27.9 eval time: 4.1
at epoch 5
train info: logloss loss:0.5320256811285776
eval info: auc:0.742, logloss:0.4964
at epoch 5 , train time: 28.1 eval time: 4.1


In [ ]:
# check the predictive performance after the model is trained
result = model.run_eval(test_file)
print(result)

In [ ]:
sb.glue("result", result)

In [ ]:
# Cleanup
tmpdir.cleanup()

## Reference
\[1\] Lian, J., Zhou, X., Zhang, F., Chen, Z., Xie, X., & Sun, G. (2018). xDeepFM: Combining Explicit and Implicit Feature Interactions for Recommender Systems. Proceedings of the 24th ACM SIGKDD International Conference on Knowledge Discovery \& Data Mining, KDD 2018, London, UK, August 19-23, 2018.<br>